<a href="https://colab.research.google.com/github/Babs-Coding/Refonte/blob/main/Final_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: 'data/coco/annotations/captions_train2017.json'

In [ ]:
!mkdir -p data/coco/annotations
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P data/coco/annotations
!unzip data/coco/annotations/annotations_trainval2017.zip -d data/coco/annotations

--2025-04-26 15:13:48--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 16.15.176.39, 3.5.30.114, 3.5.27.59, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|16.15.176.39|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘data/coco/annotations/annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  96.2MB/s    in 2.5s    

2025-04-26 15:13:51 (96.2 MB/s) - ‘data/coco/annotations/annotations_trainval2017.zip’ saved [252907541/252907541]

Archive:  data/coco/annotations/annotations_trainval2017.zip
  inflating: data/coco/annotations/annotations/instances_train2017.json  
  inflating: data/coco/annotations/annotations/instances_val2017.json  
  inflating: data/coco/annotations/annotations/captions_train2017.json  
  inflating: data/coco/annotations/annotations/captions_val2017.json  
  infl

In [ ]:
# Download using pycocotools
!pip install pycocotools
!pip install diffusers transformers torch



import zipfile
import os
import json
from pycocotools.coco import COCO
import requests

dataDir = 'data/coco'
dataType = 'train2017'
zip_path = f'{dataDir}/annotations/annotations_trainval2017.zip'
extracted_path = f'{dataDir}/annotations'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

json_file = os.path.join(extracted_path, 'annotations', f'instances_{dataType}.json')
with open(json_file, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

coco = COCO()
coco.dataset = dataset
coco.createIndex()

catIds = coco.getCatIds()
imgIds = coco.getImgIds()
images = coco.loadImgs(imgIds[:10])

os.makedirs(f"{dataDir}/images", exist_ok=True)

for img in images:
    url = img['coco_url']
    img_data = requests.get(url).content
    with open(f"{dataDir}/images/{img['file_name']}", 'wb') as handler:
        handler.write(img_data)

from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to("cuda")

prompt = "a futuristic city with flying cars"
image = pipe(prompt).images[0]
image.save("output.png")


git clone https://github.com/THUDM/CogVideo.git
cd CogVideo
conda create -n cogvideo python=3.8
conda activate cogvideo
pip install -r requirements.txt

#Generates cogvideo line above
python inference_cogvideo.py \
  --text "a panda riding a bike in the city" \
  --save_path "outputs/"







creating index...
index created!


In [ ]:
import zipfile

zip_path = 'data/coco/annotations/annotations_trainval2017.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    print("Contents of the ZIP file:")
    zip_ref.printdir()

Contents of the ZIP file:
File Name                                             Modified             Size
annotations/instances_train2017.json           2017-09-01 19:02:24    469785474
annotations/instances_val2017.json             2017-09-01 19:02:32     19987840
annotations/captions_train2017.json            2017-09-01 19:04:56     91865115
annotations/captions_val2017.json              2017-09-01 19:04:58      3872473
annotations/person_keypoints_train2017.json    2017-09-01 19:04:32    238884731
annotations/person_keypoints_val2017.json      2017-09-01 19:04:38     10020657


In [ ]:
# Mobile dev

!pip install diffusers transformers accelerate
!pip install modelscope


from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

prompt = "A fantasy landscape with floating islands and waterfalls"
image = pipe(prompt).images[0]
image.save("generated_image.png")

from modelscope.pipelines import pipeline
from modelscope.models import Model

pipe = pipeline('text-to-video-synthesis', model='damo/text-to-video-synthesis')
video_path = pipe({'text': 'A panda eating bamboo'})['output_video']






In [ ]:
#Fast API backend
!pip install fastapi uvicorn torch transformers diffusers
!pip install fastapi[all]


from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from diffusers import StableDiffusionPipeline
import torch
import os
import shutil


app = FastAPI()


pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipe.to("cuda")


class TextInput(BaseModel):
    prompt: str
    media_type: str

# Route for generating media (image or video)
@app.post("/generate/")
async def generate_media(data: TextInput):
    if data.media_type not in ['image', 'video']:
        raise HTTPException(status_code=400, detail="Unsupported media type")

    # Generate the media based on the input text
    if data.media_type == "image":
        return await generate_image(data.prompt)
    elif data.media_type == "video":
        return await generate_video(data.prompt)


async def generate_image(prompt: str):
    try:
        image = pipe(prompt).images[0]
        file_path = f"static/{prompt[:10]}.png"
        image.save(file_path)
        return {"image_url": file_path}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error generating image: {str(e)}")

# Placeholder
async def generate_video(prompt: str):
    try:
        video_path = f"static/{prompt[:10]}.mp4"
        # Simulating video generation here
        shutil.copy("sample_video.mp4", video_path)  n
        return {"video_url": video_path}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error generating video: {str(e)}")

from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all origins for now
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


# Run the server (test when coping back to colab)
# uvicorn app:app --reload





In [ ]:
#Frotend using React

npx create-react-app generative-ai-app
cd generative-ai-app
npm start

npm install axios

import React, { useState } from "react";
import axios from "axios";

function App() {
  const [prompt, setPrompt] = useState("");
  const [mediaType, setMediaType] = useState("image");
  const [resultUrl, setResultUrl] = useState("");

  const handleSubmit = async (e) => {
    e.preventDefault();
    try {
      const response = await axios.post("http://localhost:8000/generate/", {
        prompt,
        media_type: mediaType,
      });
      if (mediaType === "image") {
        setResultUrl(response.data.image_url);
      } else {
        setResultUrl(response.data.video_url);
      }
    } catch (error) {
      console.error("Error generating media:", error);
    }
  };

  return (
    <div className="App">
      <h1>Generative AI System</h1>
      <form onSubmit={handleSubmit}>
        <label>
          Prompt:
          <input
            type="text"
            value={prompt}
            onChange={(e) => setPrompt(e.target.value)}
            required
          />
        </label>
        <br />
        <label>
          Media Type:
          <select
            value={mediaType}
            onChange={(e) => setMediaType(e.target.value)}
          >
            <option value="image">Image</option>
            <option value="video">Video</option>
          </select>
        </label>
        <br />
        <button type="submit">Generate</button>
      </form>

      {resultUrl && (
        <div>
          <h2>Generated {mediaType}</h2>
          {mediaType === "image" ? (
            <img src={`http://localhost:8000/${resultUrl}`} alt="Generated" />
          ) : (
            <video width="500" controls>
              <source src={`http://localhost:8000/${resultUrl}`} type="video/mp4" />
            </video>
          )}
        </div>
      )}
    </div>
  );
}

export default App;

#Frontend should be running on localhost:3000 and backend on localhost:8000


In [ ]:
# Cloud infrastructure

# # docker builder in vs code

# # cd my project path is here

# # Set environment variables
# ENV PYTHONDONTWRITEBYTECODE=1
# ENV PYTHONUNBUFFERED=1

# # Install system packages
# RUN apt-get update && apt-get install -y \
#     python3 python3-pip git curl && \
#     rm -rf /var/lib/apt/lists/*

# # Create app directory
# FinalAssignmentCI/app

# COPY dependencies.txt
# RUN pip3 install --no-cache-dir -r dependencies.txt

# CMD ["uvicorn", "*put finalnamehere*.main:app", "--host", "0.0.0.0", "--port", "80"]


# docker build -t gen-ai-backend .
# docker run -p 8000:80 gen-ai-backend






In [ ]:
# .github/Babs-Coding/Refonte/deploy.yml
#
# name: Deploy to Cloud

# on:
  # push:
    # branches: [ main ]

# jobs:
  # build-and-deploy:
    # runs-on: ubuntu-latest
    # steps:
      # - uses: actions/checkout@v3
      # - name: Build Docker Image
        # run: docker build -t my-ai-app .
      # - name: Push to AWS ECR / Deploy to EC2
        # run: |
          # your deployment script here


In [ ]:
# CI/CD for backend FastAPI

# name: Deploy FastAPI to Heroku

# on:
#   push:
#     branches:
#       - master  # Trigger deployment on master branch push

# jobs:
#   deploy:
#     runs-on: ubuntu-latest

#     steps:
#     - name: Checkout code
#       uses: actions/checkout@v2

#     - name: Set up Python
#       uses: actions/setup-python@v2
#       with:
#         python-version: '3.8'  # Use your Python version

#     - name: Install dependencies
#       run: |
#         python -m pip install --upgrade pip
#         pip install -r requirements.txt

#     - name: Login to Heroku
#       uses: akshnz/heroku-login-action@v1
#       with:
#         email: ${{ secrets.HEROKU_EMAIL }}
#         api_key: ${{ secrets.HEROKU_API_KEY }}

#     - name: Deploy to Heroku
#       run: |
#         git remote add heroku https://git.heroku.com/${{ secrets.HEROKU_APP_NAME }}.git
#         git push heroku master



In [ ]:
#


# mkdir -p .github/Babs-Coding/Refonte
# touch .github/Babs-Coding/Refonte/deploy.yml
# touch .github/Babs-Coding/Refonte/requirements.txt

# name: Deploy React to Vercel

# on:
#   push:
#     branches:
#       - master

# jobs:
#   deploy:
#     runs-on: ubuntu-latest

#     steps:
#     - name: Checkout code
#       uses: actions/checkout@v2

#     - name: Set up Node.js
#       uses: actions/setup-node@v2
#       with:
#         node-version: '14'

#     - name: Install dependencies
#       run: npm install

#     - name: Build the app
#       run: npm run build

#     - name: Deploy to Vercel
#       uses: amondnet/vercel-action@v20
#       with:
#         vercel_token: ${{ secrets.VERCEL_TOKEN }}
#         vercel_project_id: ${{ secrets.VERCEL_PROJECT_ID }}
#         vercel_org_id: ${{ secrets.VERCEL_ORG_ID }}
#         vercel_git_branch: ${{ github.ref }}
#       env:
#         VERCEL_TOKEN: ${{ secrets.VERCEL_TOKEN }}


